In [1]:
from datetime import datetime as dt

import numpy as np
import pandas as pd
import sounderpy as spy

from ecape.calc import calc_ecape
from metpy.units import units
from metpy import calc as mpcalc


## ---------------------------------- SOUNDERPY ----------------------------------- ##
##          Vertical Profile Data Retrieval and Analysis Tool For Python            ##
##                      v3.0.8 | Jan 2025 | (C) Kyle J Gillett                      ##
##                 Docs: https://kylejgillett.github.io/sounderpy/                  ##
## --------------------- THANK YOU FOR USING THIS PACKAGE! ------------------------ ##



In [2]:
date_i_want = dt(2022, 6, 2, 0, 0)

In [3]:
acars_all_list = []
airports_i_want = {'IAH' : {'latitude' : 29.9844353, 'longitude' : -95.3414425},
                   'HOU' : {'latitude' : 29.6457998, 'longitude' : -95.2772316},
                   'LBX' : {'latitude' : 29.1086389, 'longitude' : -95.4620833},
                   'GLS' : {'latitude' : 29.2653333, 'longitude' : -94.8604167},
                   'EFD' : {'latitude' : 29.6073333, 'longitude' : -95.1587500},
                   'IWS' : {'latitude' : 29.8181944, 'longitude' : -95.6726111},
                   'SGR' : {'latitude' : 29.6222486, 'longitude' : -95.6565342},
                   'DWH' : {'latitude' : 30.0617791, 'longitude' : -95.5527884},
                   'CXO' : {'latitude' : 30.3533955, 'longitude' : -95.4150819},
                   'HPY' : {'latitude' : 29.7860833, 'longitude' : -94.9526667}
                   }
for hour_to_pull in range(24):
    acars_conn = spy.acars_data(str(date_i_want.year).zfill(4), str(date_i_want.month).zfill(2), str(date_i_want.day).zfill(2), str(hour_to_pull).zfill(2))
    this_hour_acars_list = [prof for prof in acars_conn.list_profiles() if prof[:3] in airports_i_want.keys()]
    acars_all_list.extend(this_hour_acars_list)

> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-06-02 00Z...
> COMPLETE --------
> RUNTIME: 00:00:00
> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-06-02 01Z...
> COMPLETE --------
> RUNTIME: 00:00:00
> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-06-02 02Z...
> COMPLETE --------
> RUNTIME: 00:00:04
> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-06-02 03Z...
> COMPLETE --------
> RUNTIME: 00:00:00
> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-06-02 04Z...
> COMPLETE --------
> RUNTIME: 00:00:00
> LIST ACARS PROFILES FUNCTION
  ---------------------------------
> AVAILABLE ACARS PROFILES FOR 2022-06-02 05Z...
> COMPLETE --------
> RUNTIME: 00:00:01
> LIST ACARS PROFILES FUNCTION
  -------------------------------

In [4]:
acars_all_list

['HOU_0030',
 'HOU_0150',
 'HOU_0250',
 'HOU_0300',
 'HOU_0340',
 'HOU_0350',
 'HOU_1200',
 'HOU_1220',
 'HOU_1230',
 'HOU_1320',
 'HOU_1350',
 'HOU_1400',
 'IAH_1820',
 'HOU_1920',
 'IAH_1930',
 'HOU_2020',
 'IAH_2040',
 'IAH_2130']

In [5]:
acars_profile_dts = []
acars_profile_lons = []
acars_profile_lats = []
acars_profile_files = []


for acars_loctime in acars_all_list:
    acars_profile_dts.append(date_i_want.replace(hour=int(acars_loctime[-4:-2]), minute=int(acars_loctime[-2:])))
    acars_profile_lats.append(airports_i_want[acars_loctime[:3]]['latitude'])
    acars_profile_lons.append(airports_i_want[acars_loctime[:3]]['longitude'])
    acars_profile_files.append('ACARS+'+acars_loctime)

In [6]:
for acars_profile_file, acars_dt in zip(acars_profile_files, acars_profile_dts):
    acars_conn = spy.acars_data(str(acars_dt.year).zfill(4), str(acars_dt.month).zfill(2), str(acars_dt.day).zfill(2), str(acars_dt.hour).zfill(2))
    sounding = acars_conn.get_profile(acars_profile_file.replace('ACARS+', ''), hush=True, clean_it=True)
    this_sonde_data = pd.DataFrame(
        {'pres' : sounding['p'].to('hPa').m,
        'tdry' : sounding['T'].to('degC').m,
        'dp' : sounding['Td'].to('degC').m,
        'u_wind' : sounding['u'].to('m/s').m,
        'v_wind' : sounding['v'].to('m/s').m,
        'alt' : sounding['z'].to('m').m}
    )
    break

> ACARS DATA ACCESS FUNCTION
  ---------------------------------


/Users/stgardner4/micromamba/envs/tracer/lib/python3.12/site-packages/sounderpy/acars_data.py:138: UserWarning: Input line 1 contained no data and will not be counted towards `max_rows=50000`.  This differs from the behaviour in NumPy <=1.22 which counted lines rather than rows.  If desired, the previous behaviour can be achieved by using `itertools.islice`.
Please see the 1.23 release notes for an example on how to do this.  If you wish to ignore this warning, use `warnings.filterwarnings`.  This warning is expected to be removed in the future and is given only once per `loadtxt` call.
  header = loadtxt(urlopen(profile_url).readlines()[0:3], dtype='str', comments="%", unpack=True)


    > COMPLETE --------
    > RUNTIME: 00:00:00


In [7]:
sounding

{'p': <Quantity([1006.81  992.31  984.78  968.34  952.58  944.38  940.87  935.9   918.12
   889.32  863.48  851.51  830.95  790.93  755.8   723.77  693.2   677.53
   667.87  645.22  619.95  593.16  570.47  550.17  531.53  514.65  497.12
   479.23  464.87  450.04  354.61  283.82], 'hectopascal')>,
 'z': <Quantity([  56.  178.  242.  383.  520.  592.  623.  667.  826. 1089. 1331. 1445.
  1644. 2043. 2407. 2751. 3091. 3270. 3382. 3650. 3958. 4296. 4592. 4865.
  5123. 5363. 5619. 5888. 6110. 6345. 8029. 9535.], 'meter')>,
 'T': <Quantity([ 28.55  27.05  26.55  25.25  23.35  22.55  22.35  21.75  21.55  20.05
   18.55  17.55  16.85  15.55  12.75  10.05   7.35   7.05   7.35   5.55
    4.05   2.05   0.85  -2.25  -3.25  -4.75  -6.25  -8.45  -9.45 -10.95
  -24.95 -37.65], 'degree_Celsius')>,
 'Td': <Quantity([ 22.47  22.18  22.19  21.3   21.39  21.13  20.85  20.77  16.93  14.84
   15.19  14.11  12.21   8.76   6.44   4.49   1.36 -15.61 -23.36 -24.35
  -25.7  -34.9  -39.63 -18.07 -19.34 -22.57 -19

In [7]:
temp = this_sonde_data['tdry'].values * units.degC
dew = this_sonde_data['dp'].values * units.degC
pressure = this_sonde_data['pres'].values * units.hPa
height = this_sonde_data['alt'].values * units.m
u = this_sonde_data['u_wind'].values * units.m/units.s
v = this_sonde_data['v_wind'].values * units.m/units.s

In [8]:
spc_hum = mpcalc.specific_humidity_from_dewpoint(pressure, dew)

In [9]:
mlecape = calc_ecape(height, pressure, temp, spc_hum, u, v, cape_type='mixed_layer')
mlcape, mlcin = mpcalc.mixed_layer_cape_cin(pressure, temp, dew)

In [10]:
sounding.shape

AttributeError: 'dict' object has no attribute 'shape'

In [90]:
sounding[0, :] # Pressure, hPa

array(['1006.81,', '992.31,', '984.78,', '968.34,', '952.58,', '944.38,',
       '940.87,', '935.90,', '918.12,', '889.32,', '863.48,', '851.51,',
       '830.95,', '790.93,', '755.80,', '723.77,', '693.20,', '677.53,',
       '667.87,', '645.22,', '619.95,', '593.16,', '570.47,', '550.17,',
       '531.53,', '514.65,', '497.12,', '479.23,', '464.87,', '450.04,',
       '354.61,', '283.82,'], dtype='<U8')

In [91]:
sounding[1, :] # Height, m

array(['56.00,', '178.00,', '242.00,', '383.00,', '520.00,', '592.00,',
       '623.00,', '667.00,', '826.00,', '1089.00,', '1331.00,',
       '1445.00,', '1644.00,', '2043.00,', '2407.00,', '2751.00,',
       '3091.00,', '3270.00,', '3382.00,', '3650.00,', '3958.00,',
       '4296.00,', '4592.00,', '4865.00,', '5123.00,', '5363.00,',
       '5619.00,', '5888.00,', '6110.00,', '6345.00,', '8029.00,',
       '9535.00,'], dtype='<U8')

In [92]:
sounding[2, :] # Temperature, C

array(['28.55,', '27.05,', '26.55,', '25.25,', '23.35,', '22.55,',
       '22.35,', '21.75,', '21.55,', '20.05,', '18.55,', '17.55,',
       '16.85,', '15.55,', '12.75,', '10.05,', '7.35,', '7.05,', '7.35,',
       '5.55,', '4.05,', '2.05,', '0.85,', '-2.25,', '-3.25,', '-4.75,',
       '-6.25,', '-8.45,', '-9.45,', '-10.95,', '-24.95,', '-37.65,'],
      dtype='<U8')

In [93]:
sounding[3, :] # Dewpoint, C

array(['22.47,', '22.18,', '22.19,', '21.30,', '21.39,', '21.13,',
       '20.85,', '20.77,', '16.93,', '14.84,', '15.19,', '14.11,',
       '12.21,', '8.76,', '6.44,', '4.49,', '1.36,', '-15.61,', '-23.36,',
       '-24.35,', '-25.70,', '-34.90,', '-39.63,', '-18.07,', '-19.34,',
       '-22.57,', '-19.24,', '-20.19,', '-26.82,', '-40.41,', '-49.16,',
       '-53.17,'], dtype='<U8')

In [94]:
sounding[4, :] # Wind speed, m/s

array(['148.00,', '154.00,', '154.00,', '160.00,', '164.00,', '164.00,',
       '165.00,', '169.00,', '165.00,', '154.00,', '139.00,', '131.00,',
       '142.00,', '122.00,', '155.00,', '100.00,', '93.00,', '156.00,',
       '108.00,', '57.00,', '334.00,', '332.00,', '333.00,', '4.00,',
       '323.00,', '315.00,', '321.00,', '299.00,', '245.00,', '259.00,',
       '298.00,', '281.00,'], dtype='<U8')

In [95]:
sounding[5, :]

array(['7.95', '11.06', '11.06', '9.89', '11.06', '9.89', '11.06',
       '11.06', '11.06', '11.06', '8.92', '7.95', '6.98', '2.91', '1.94',
       '2.91', '5.04', '6.01', '0.97', '6.01', '6.01', '6.98', '6.01',
       '6.98', '2.91', '2.91', '4.07', '6.01', '6.01', '8.92', '20.95',
       '19.98'], dtype='<U8')